### 三层继承结构
+ 三层继承结构使代码得到最大程度的复用，并且使得添加内容更加简单
+ 如下图为常见的电商页面
![title](img/常见电商页面(三层继承结构).png)

In [ ]:
# 1.创建根级模板
# 名称为“base.html”
# 存放整个站点共用的内容
<!DOCTYPE html>
<html>
<head>
    <title>{%block title%}{%endblock%} 水果超市</title>
</head>
<body>
top--{{logo}}
<hr/>
{%block left%}{%endblock%}
{%block content%}{%endblock%}
<hr/>
bottom
</body>
</html>

In [1]:
# 2.创建分支模版
# 继承自base.html
# 名为“base_*.html”
# 定义特定分支共用的内容
# 定义base_goods.html
{%extends 'temtest/base.html'%}
{%block title%}商品{%endblock%}
{%block left%}
<h1>goods left</h1>
{%endblock%}
定义base_user.html
{%extends 'temtest/base.html'%}
{%block title%}用户中心{%endblock%}
{%block left%}
<font color='blue'>user left</font>
{%endblock%}
定义index.html，继承自base.html，不需要写left块
{%extends 'temtest/base.html'%}
{%block content%}
首页内容
{%endblock content%}

SyntaxError: invalid syntax (<ipython-input-1-82d4e47113bc>, line 6)

In [ ]:
# 3.为具体页面创建模板，继承自分支模板
# 定义商品列表页goodslist.html
{%extends 'temtest/base_goods.html'%}
{%block content%}
商品正文列表
{%endblock content%}
定义用户密码页userpwd.html
{%extends 'temtest/base_user.html'%}
{%block content%}
用户密码修改
{%endblock content%}

In [ ]:
# 4.视图调用具体页面，并传递模板中需要的数据
# 首页视图index
logo='welcome to itcast'
def index(request):
    return render(request, 'temtest/index.html', {'logo': logo})
商品列表视图goodslist
def goodslist(request):
    return render(request, 'temtest/goodslist.html', {'logo': logo})
用户密码视图userpwd
def userpwd(request):
    return render(request, 'temtest/userpwd.html', {'logo': logo})

In [ ]:
# 5.配置url
from django.conf.urls import url
from . import views
urlpatterns = [
    url(r'^$', views.index, name='index'),
    url(r'^list/$', views.goodslist, name='list'),
    url(r'^pwd/$', views.userpwd, name='pwd'),
]

### 验证码
+ 在用户注册、登录页面，为了防止暴力请求，可以加入验证码功能，如果验证码错误，则不需要继续处理，可以减轻一些服务器的压力
+ 使用验证码也是一种有效的防止crsf的方法<br><br>

#### 验证码视图
+ 新建viewsUtil.py，定义函数verifycode
+ 此段代码用到了PIL中的Image、ImageDraw、ImageFont模块，需要先安装Pillow（3.4.1）包，详细文档参考http://pillow.readthedocs.io/en/3.4.x/
+ Image表示画布对象
+ ImageDraw表示画笔对象
+ ImageFont表示字体对象，ubuntu的字体路径为“/usr/share/fonts/truetype/freefont”

In [ ]:
from django.http import HttpResponse
def verifycode(request):
    #引入绘图模块
    from PIL import Image, ImageDraw, ImageFont
    #引入随机函数模块
    import random
    #定义变量，用于画面的背景色、宽、高
    bgcolor = (random.randrange(20, 100), random.randrange(
        20, 100), 255)
    width = 100
    height = 25
    #创建画面对象
    im = Image.new('RGB', (width, height), bgcolor)
    #创建画笔对象
    draw = ImageDraw.Draw(im)
    #调用画笔的point()函数绘制噪点
    for i in range(0, 100):
        xy = (random.randrange(0, width), random.randrange(0, height))
        fill = (random.randrange(0, 255), 255, random.randrange(0, 255))
        draw.point(xy, fill=fill)
    #定义验证码的备选值
    str1 = 'ABCD123EFGHIJK456LMNOPQRS789TUVWXYZ0'
    #随机选取4个值作为验证码
    rand_str = ''
    for i in range(0, 4):
        rand_str += str1[random.randrange(0, len(str1))]
    #构造字体对象
    font = ImageFont.truetype('FreeMono.ttf', 23)
    #构造字体颜色
    fontcolor = (255, random.randrange(0, 255), random.randrange(0, 255))
    #绘制4个字
    draw.text((5, 2), rand_str[0], font=font, fill=fontcolor)
    draw.text((25, 2), rand_str[1], font=font, fill=fontcolor)
    draw.text((50, 2), rand_str[2], font=font, fill=fontcolor)
    draw.text((75, 2), rand_str[3], font=font, fill=fontcolor)
    #释放画笔
    del draw
    #存入session，用于做进一步验证
    request.session['verifycode'] = rand_str
    #内存文件操作
    import cStringIO
    buf = cStringIO.StringIO()
    #将图片保存在内存中，文件类型为png
    im.save(buf, 'png')
    #将内存中的图片数据返回给客户端，MIME类型为图片png
    return HttpResponse(buf.getvalue(), 'image/png')

In [ ]:
# 配置url
# 在urls.py中定义请求验证码视图的url
from . import viewsUtil

urlpatterns = [
    url(r'^verifycode/$', viewsUtil.verifycode),
]

In [ ]:
# 启动服务器，查看显示成功
# 扩展：点击“看不清，换一个”时，可以换一个新的验证码
<script type="text/javascript" src="/static/jquery-1.12.4.min.js"></script>
<script type="text/javascript">
    $(function(){
        $('#verifycodeChange').css('cursor','pointer').click(function() {
            $('#verifycode').attr('src',$('#verifycode').attr('src')+1)
        });
    });
</script>
<img id='verifycode' src="/verifycode/?1" alt="CheckCode"/>
<span id='verifycodeChange'>看不清，换一个</span>

# 为了能够实现提交功能，需要增加form和input标签
<form method='post' action='/verifycodeValid/'>
    <input type="text" name="vc">
    <img id='verifycode' src="/verifycode/?1" alt="CheckCode"/>
<span id='verifycodeChange'>看不清，换一个</span>
<br>
<input type="submit" value="提交">
</form>

In [ ]:
# 验证
# 接收请求的信息，与session中的内容对比
from django.http import HttpResponse

def verifycodeValid(request):
    vc = request.POST['vc']
    if vc.upper() == request.session['verifycode']:
        return HttpResponse('ok')
    else:
        return HttpResponse('no')
# 配置验证处理的url
urlpatterns = [
    url(r'^verifycodeValid/$', views.verifycodeValid),
]